In [ ]:
import pathlib
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

from datetime import datetime, timedelta
import time
import json
import hashlib
import re
import pandas as pd

import requests
import urllib
from bs4 import BeautifulSoup

import pickle
url = 'https://infinite.tcgplayer.com/article/Pok%C3%A9mon-TCG-Silver-Tempest-Pull-Rates/6490d591-e582-4930-8446-00e190876d30/'
timeout = 10

chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(executable_path='./chromedriver.exe', options=chrome_options)

browser.get(url)
time.sleep(timeout)

source = browser.page_source.encode("utf-8")
soup = BeautifulSoup(source)
dfRate = pd.read_html(str(soup.find('div', {'class' : 'article-body'}).find('table')))
dfRate = pd.DataFrame(dfRate[0])
print(type(dfRate))
print(dfRate)
#Pack  Cost down below this comment 
urlTempestBoosterPack = 'https://www.tcgplayer.com/product/283388/pokemon-swsh12-silver-tempest-silver-tempest-booster-pack?Language=English&page=1'

timeout1 = 10
# launch browser
chrome_options = Options()
chrome_options.add_argument("--headless")
browser1 = webdriver.Chrome(executable_path='./chromedriver.exe', options=chrome_options)

browser1.get(urlTempestBoosterPack)
time.sleep(timeout)

sourceTempest = browser1.page_source.encode("utf-8")
soupTempest = BeautifulSoup(sourceTempest)

priceTempest = float(soupTempest.find('span', {'class' : 'spotlight__price'}).text[1:])

aTempest = soupTempest.find('section', {'class' : 'product-details__listings-total'}).find('h2').text
num_for_saleTempest = int(re.findall(r'\d+', aTempest)[0])

# browser.close()


print(priceTempest)

print(num_for_saleTempest)
#do the main as a function after
def update_TRAINERG(row):
    if (row['Number'] >=  23) and (row['Number'] <= 28):
        row['Rarity'] = 'Trainer'
    
    return row
def update_rainbow(row):
    if (row['Left'] >  195) and (row['Left'] < 208):
        row['Rarity'] = 'Rainbow Rare'
    
    return row

def update_FULL_TRAINER(row):
    if (row['Left'] >=  188) and (row['Left'] <= 195):
        row['Rarity'] = 'Full-Art Trainer'
    
    return row

def update_gold(row):
    if row['Left'] >= 208 :
        row['Rarity'] = 'Gold Rare'
    
    return row
def update_V(row):
    if row['TYPE'] == "V" :
        row['Rarity'] = 'V'
    
    return row
def update_VMAX(row):
    if row['TYPE'] == "VMAX" :
        row['Rarity'] = 'VMAX'
    
    return row
def update_VSTAR(row):
    if row['TYPE'] == "VSTAR" :
        row['Rarity'] = 'VSTAR'
    
    return row
def update_FULLART(row):
    if row['TYPE'] == "FULL ART" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_ART(row):
    if row['ART'] == "Art" :
        row['Rarity'] = 'Full-Art'
    
    return row

def update_ALT(row):
    if row['ART'] == "Alternate Full Art" :
        row['Rarity'] = 'Alt-Art V'
    
    return row

def update_FULL_ART(row):
    if row['ART'] == "Full Art" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_FULL_ART2(row):
    if row['ART'] == "V Full Art" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_VMAX2(row):
    if row['TYPE'] == "VMAX":
        row['Rarity'] = 'VMAX'
    
    return row
def update_SECRET(row):
    if row['Rarity'] == "Secret Rare" :
        row['TYPE'] = 'Gold-and-Black'
    
    return row
def update_GOLD_AND_BLACK(row):
    if row['TYPE'] == "Gold-and-Black" :
        row['Rarity'] = 'Gold-and-Black'
    
    return row
def update_V2(row):
    if row['TYPE'] == "V" :
        row['Rarity'] = 'V-Rare'
    else:
        row["Rarity"] = "Non-V"

    
    return row
def update_TrainerPulls(df):
    num = 8
    num2 = 1
    #change this to concat ltr 
    if not df.loc[df["NAME"] == "VTrainer", "NAME"].empty:
        df = df.append({"Card Type": "VTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
        num -= 1
    
    if not df.loc[df["TYPE"] == "VMAXTrainer", "TYPE"].empty:
        num -= 1 
        df = df.append({"Card Type": "VMAXTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
        
    
    if not df.loc[df["OTHER"] == "Trainer", "OTHER"].empty:
        df = df.append({"Card Type": "Trainer", "PullRate": df["PullRate"][num] / 3.0 }, ignore_index=True)
    if not df.loc[df["NAME"] == "VMAX", "NAME"].empty:
        
        df = df.append({"Card Type": "VMAX", "PullRate": df["PullRate"][num2] / 2.0 }, ignore_index=True)
        
    if not df.loc[df["TYPE"] == "VSTAR", "TYPE"].empty:
        
        df = df.append({"Card Type": "VSTAR", "PullRate": df["PullRate"][num2] / 2 }, ignore_index=True)
        
    
        
        # Might want to change this function to something like this df.loc[len(df.index)] = ['Amy', 89, 93]  adds to the existing df without naming the columns 
    
    return df
def drop_TRAINERGALLARY(df):
    df = df[df["Card Type"] != "Secret Rare"]
    df = df[df["Card Type"] != "Trainer Gallery"]
    df = df[df["Card Type"] != "Ultra Rare"]
    return df

data = pd.read_html("https://shop.tcgplayer.com/price-guide/pokemon/swsh12-silver-tempest")
print(data)
print(type(data))
df = pd.DataFrame(data[0])
print(df)


df["Market Price"] = df["Market Price"].str.replace("$", "",regex = True )
df.drop(df[df["Market Price"] == '—'].index, inplace = True) 
print(df['Market Price'].to_list())
df["Price"] = df["Market Price"].astype(float)
print(type(df["Price"][1]))
print(df)


df.drop(labels = "Unnamed: 5", axis = 1, inplace = True)

df.drop(labels = "Listed Median", axis = 1, inplace = True)
df.drop(labels = "Market Price", axis = 1, inplace = True)

print(df)
#Dont do splits I want to keep Full arts
df["PRODUCT"] = df["PRODUCT"].str.split(")").str[0]
df[["Product", "Art Type"]] = df["PRODUCT"].str.split("(", expand = True)
#df["PRODUCT"] = df["PRODUCT"].str.split("(").str[0]
df["Product"] = df["Product"].str.strip()
#df["PRODUCT"] = df["PRODUCT"].str.strip()
df[["Left", "Right"]] = df["Number"].str.split("/",expand = True)
df["Left"] = df["Left"].astype(float)
print(df)



print(df)
df = df.drop(labels = "Product" , axis = 1)
df = df.drop(labels = "Art Type" , axis = 1)
df["PRODUCT"] = df["PRODUCT"].str.replace("Alolan ", "", regex = True)
df[[ 'NAME', 'TYPE', 'ART']] = df['PRODUCT'].str.split(' ', n = 2,  expand=True) # dont do this its dumb 
df["ART"] = df["ART"].str.replace("(", "", regex = True)
df["TYPE"] = df["TYPE"].str.replace("(", "", regex = True)
df = df.apply(update_V, axis=1)
df = df.apply(update_VMAX, axis=1)
df = df.apply(update_VSTAR, axis=1)
df = df.apply(update_FULLART, axis=1)
df = df.apply(update_ART, axis=1)
df = df.apply(update_rainbow, axis=1)
df = df.apply(update_gold, axis=1)
df = df.apply(update_ALT, axis=1)
df = df.apply(update_FULL_ART, axis = 1)
df = df.apply(update_FULL_ART2, axis = 1)
df = df.apply(update_FULL_TRAINER, axis = 1)
df["PRODUCT"] = df["PRODUCT"].str.replace("(", "", regex = True)
df["PRODUCT"] = df["PRODUCT"].str.strip()
df = df.sort_values("Price")
print(df)
uValues = df['Rarity'].unique()
print(uValues)
df.drop(labels = "ART", axis = 1, inplace = True)
df.drop(labels = "NAME", axis = 1, inplace = True)
df.drop(labels = "TYPE", axis = 1, inplace = True)
df.drop(labels = "Number", axis = 1, inplace = True)
df.drop(labels = "Left", axis = 1, inplace = True)
df.drop(labels = "Right", axis = 1, inplace = True)


print(df)
avgPrice = df.groupby("Rarity").mean()
print(avgPrice)
avgPrice.reset_index( inplace = True)
print(avgPrice)
dataSubset = df[df["Price"] > priceTempest]
print(dataSubset)


data1 = pd.read_html("https://shop.tcgplayer.com/price-guide/pokemon/swsh12-silver-tempest-trainer-gallery")
print(data1)
print(type(data1))
dfTrainer = pd.DataFrame(data1[0])
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.split(")").str[0]
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.split("(").str[0]
print(dfTrainer)
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.strip()
dfTrainer[[ 'NAME', 'TYPE']] = dfTrainer['PRODUCT'].str.split(' ', n = 1,  expand=True)
dfTrainer["TYPE"] = dfTrainer["TYPE"].str.strip()
dfTrainer = dfTrainer.apply(update_SECRET, axis=1)
print(dfTrainer)
dfTrainer = dfTrainer.apply(update_V2, axis=1)
dfTrainer = dfTrainer.apply(update_VMAX2, axis=1)
dfTrainer = dfTrainer.apply(update_GOLD_AND_BLACK, axis = 1)


print(dfTrainer)
dfTrainer["Number"] = dfTrainer["Number"].str.split("/").str[0]

dfTrainer["Number"] = dfTrainer["Number"].str.replace("TG", "", regex = True)
dfTrainer["Number"] = dfTrainer["Number"].astype(int)
dfTrainer = dfTrainer.apply(update_TRAINERG, axis=1)
dfTrainer.drop(labels = "Number", axis = 1, inplace = True)
dfTrainer.drop(labels = "NAME", axis = 1, inplace = True)
dfTrainer.drop(labels = "TYPE", axis = 1, inplace = True)
dfTrainer.drop(labels = "Unnamed: 5", axis = 1, inplace = True)
dfTrainer.drop(labels = "Listed Median", axis = 1, inplace = True)
dfTrainer.rename(columns = {'Market Price':'Price'}, inplace = True)
dfTrainer["Price"] = dfTrainer["Price"].str.replace("$", "", regex = True)
dfTrainer["Price"] = dfTrainer["Price"].astype(float)
print(dfTrainer)

#Working on thhis dataframe 
dfRate.rename(columns = {'Pull Rate (%)' : 'PullRate'}, inplace = True)
dfRate["PullRate"] = dfRate["PullRate"].str.replace("%", "", regex = True)
dfRate["PullRate"] = dfRate["PullRate"].str.strip()
dfRate.drop(labels = "Pull Rate (Per X Packs)", axis = 1, inplace = True)
dfRate = drop_TRAINERGALLARY(dfRate)
print(dfRate)

dfRate["Card Type"] = dfRate["Card Type"].str.replace("Pokémon ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Pokémon", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.strip()
print(dfRate)
dfRate["PullRate"] = dfRate["PullRate"].astype(float)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Normal ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace(",", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("or ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("V VMAX", "VTrainer VMAXTrainer", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Gold-and-Black VMAX", "Gold-and-Black", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.strip()
dfRate[[ 'NAME', 'TYPE', "OTHER"]] = dfRate['Card Type'].str.split(' ', n = 2,  expand=True)
dfRate = update_TrainerPulls(dfRate)
dfRate.drop(labels = "NAME", axis = 1, inplace = True)
dfRate.drop(labels = "TYPE", axis = 1, inplace = True)
dfRate.drop(labels = "OTHER", axis = 1, inplace = True)
dfRate.drop(1, inplace = True)
dfRate.drop(6, inplace = True)
dfRate.reset_index( inplace = True)
dfRate.drop(labels = "index", axis = 1, inplace = True)
print(dfRate)
dfRate.rename(columns = {'Card Type':'Rarity'}, inplace = True)
dfMerged = dfRate.merge(dfTrainer, on='Rarity')
dfMerged['Weigted'] = dfMerged['PullRate'] * dfMerged['Price']
print(dfMerged)
# dfRate["PullRate"][8] = dfRate["PullRate"][8] / 3


#change the rairtys in trainer set sooo do it ltr


#sort by values to check how Alternative ART looks like which column it falls 
#make a another function that overlaps Full Art with that checks with ART so it doesnt miss any 
#ove Vstar into a different rarity etcc.... 
#When doing the math its 6 commons and 3 uncommons one rare but the last 2 cards rare 

In [7]:
import pathlib
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

from datetime import datetime, timedelta
import time
import json
import hashlib
import re
import pandas as pd

import requests
import urllib
from bs4 import BeautifulSoup

import pickle
url = 'https://infinite.tcgplayer.com/article/Pok%C3%A9mon-TCG-Silver-Tempest-Pull-Rates/6490d591-e582-4930-8446-00e190876d30/'
timeout = 10

chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(executable_path='./chromedriver.exe', options=chrome_options)

browser.get(url)
time.sleep(timeout)

source = browser.page_source.encode("utf-8")
soup = BeautifulSoup(source)
dfRate = pd.read_html(str(soup.find('div', {'class' : 'article-body'}).find('table')))
dfRate = pd.DataFrame(dfRate[0])
print(type(dfRate))
print(dfRate)
#Pack  Cost down below this comment 
urlTempestBoosterPack = 'https://www.tcgplayer.com/product/283388/pokemon-swsh12-silver-tempest-silver-tempest-booster-pack?Language=English&page=1'

timeout1 = 10
# launch browser
chrome_options = Options()
chrome_options.add_argument("--headless")
browser1 = webdriver.Chrome(executable_path='./chromedriver.exe', options=chrome_options)

browser1.get(urlTempestBoosterPack)
time.sleep(timeout)

sourceTempest = browser1.page_source.encode("utf-8")
soupTempest = BeautifulSoup(sourceTempest)

priceTempest = float(soupTempest.find('span', {'class' : 'spotlight__price'}).text[1:])

aTempest = soupTempest.find('section', {'class' : 'product-details__listings-total'}).find('h2').text
num_for_saleTempest = int(re.findall(r'\d+', aTempest)[0])

# browser.close()


print(priceTempest)

print(num_for_saleTempest)
#do the main as a function after
def update_TRAINERG(row):
    if (row['Number'] >=  23) and (row['Number'] <= 28):
        row['Rarity'] = 'Trainer'
    
    return row
def update_rainbow(row):
    if (row['Left'] >  195) and (row['Left'] < 208):
        row['Rarity'] = 'Rainbow Rare'
    
    return row

def update_FULL_TRAINER(row):
    if (row['Left'] >=  188) and (row['Left'] <= 195):
        row['Rarity'] = 'Full-Art Trainer'
    
    return row

def update_gold(row):
    if row['Left'] >= 208 :
        row['Rarity'] = 'Gold Rare'
    
    return row
def update_V(row):
    if row['TYPE'] == "V" :
        row['Rarity'] = 'V'
    
    return row
def update_VMAX(row):
    if row['TYPE'] == "VMAX" :
        row['Rarity'] = 'VMAX'
    
    return row
def update_VSTAR(row):
    if row['TYPE'] == "VSTAR" :
        row['Rarity'] = 'VSTAR'
    
    return row
def update_FULLART(row):
    if row['TYPE'] == "FULL ART" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_ART(row):
    if row['ART'] == "Art" :
        row['Rarity'] = 'Full-Art'
    
    return row

def update_ALT(row):
    if row['ART'] == "Alternate Full Art" :
        row['Rarity'] = 'Alt-Art V'
    
    return row

def update_FULL_ART(row):
    if row['ART'] == "Full Art" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_FULL_ART2(row):
    if row['ART'] == "V Full Art" :
        row['Rarity'] = 'Full-Art'
    
    return row
def update_VMAX2(row):
    if row['TYPE'] == "VMAX":
        row['Rarity'] = 'VMAX'
    
    return row
def update_SECRET(row):
    if row['Rarity'] == "Secret Rare" :
        row['TYPE'] = 'Gold-and-Black'
    
    return row
def update_GOLD_AND_BLACK(row):
    if row['TYPE'] == "Gold-and-Black" :
        row['Rarity'] = 'Gold-and-Black'
    
    return row
def update_V2(row):
    if row['TYPE'] == "V" :
        row['Rarity'] = 'V-Rare'
    else:
        row["Rarity"] = "Non-V"

    
    return row
def update_TrainerPulls(df):
    num = 8
    num2 = 1
    #change this to concat ltr 
    if not df.loc[df["NAME"] == "VTrainer", "NAME"].empty:
        df = df.append({"Card Type": "VTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
        num -= 1
    
    if not df.loc[df["TYPE"] == "VMAXTrainer", "TYPE"].empty:
        num -= 1 
        df = df.append({"Card Type": "VMAXTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
        
    
    if not df.loc[df["OTHER"] == "Trainer", "OTHER"].empty:
        df = df.append({"Card Type": "Trainer", "PullRate": df["PullRate"][num] / 3.0 }, ignore_index=True)
    if not df.loc[df["NAME"] == "VMAX", "NAME"].empty:
        
        df = df.append({"Card Type": "VMAX", "PullRate": df["PullRate"][num2] / 2.0 }, ignore_index=True)
        
    if not df.loc[df["TYPE"] == "VSTAR", "TYPE"].empty:
        
        df = df.append({"Card Type": "VSTAR", "PullRate": df["PullRate"][num2] / 2 }, ignore_index=True)
        
    
        
        # Might want to change this function to something like this df.loc[len(df.index)] = ['Amy', 89, 93]  adds to the existing df without naming the columns 
    
    return df
def drop_TRAINERGALLARY(df):
    df = df[df["Card Type"] != "Secret Rare"]
    df = df[df["Card Type"] != "Trainer Gallery"]
    df = df[df["Card Type"] != "Ultra Rare"]
    return df

data = pd.read_html("https://shop.tcgplayer.com/price-guide/pokemon/swsh12-silver-tempest")
print(data)
print(type(data))
df = pd.DataFrame(data[0])
print(df)


df["Market Price"] = df["Market Price"].str.replace("$", "",regex = True )
df.drop(df[df["Market Price"] == '—'].index, inplace = True)
print(df['Market Price'].to_list())
df["Price"] = df["Market Price"].astype(float)
print(type(df["Price"][1]))
print(df)


df.drop(labels = "Unnamed: 5", axis = 1, inplace = True)

df.drop(labels = "Listed Median", axis = 1, inplace = True)
df.drop(labels = "Market Price", axis = 1, inplace = True)

print(df)
#Dont do splits I want to keep Full arts
df["PRODUCT"] = df["PRODUCT"].str.split(")").str[0]
df[["Product", "Art Type"]] = df["PRODUCT"].str.split("(", expand = True)
#df["PRODUCT"] = df["PRODUCT"].str.split("(").str[0]
df["Product"] = df["Product"].str.strip()
#df["PRODUCT"] = df["PRODUCT"].str.strip()
df[["Left", "Right"]] = df["Number"].str.split("/",expand = True)
df["Left"] = df["Left"].astype(float)
print(df)



print(df)
df = df.drop(labels = "Product" , axis = 1)
df = df.drop(labels = "Art Type" , axis = 1)
df["PRODUCT"] = df["PRODUCT"].str.replace("Alolan ", "", regex = True)
df[[ 'NAME', 'TYPE', 'ART']] = df['PRODUCT'].str.split(' ', n = 2,  expand=True) # dont do this its dumb 
df["ART"] = df["ART"].str.replace("(", "", regex = True)
df["TYPE"] = df["TYPE"].str.replace("(", "", regex = True)
df = df.apply(update_V, axis=1)
df = df.apply(update_VMAX, axis=1)
df = df.apply(update_VSTAR, axis=1)
df = df.apply(update_FULLART, axis=1)
df = df.apply(update_ART, axis=1)
df = df.apply(update_rainbow, axis=1)
df = df.apply(update_gold, axis=1)
df = df.apply(update_ALT, axis=1)
df = df.apply(update_FULL_ART, axis = 1)
df = df.apply(update_FULL_ART2, axis = 1)
df = df.apply(update_FULL_TRAINER, axis = 1)
df["PRODUCT"] = df["PRODUCT"].str.replace("(", "", regex = True)
df["PRODUCT"] = df["PRODUCT"].str.strip()
df = df.sort_values("Price")
print(df)
uValues = df['Rarity'].unique()
print(uValues)
df.drop(labels = "ART", axis = 1, inplace = True)
df.drop(labels = "NAME", axis = 1, inplace = True)
df.drop(labels = "TYPE", axis = 1, inplace = True)
df.drop(labels = "Number", axis = 1, inplace = True)
df.drop(labels = "Left", axis = 1, inplace = True)
df.drop(labels = "Right", axis = 1, inplace = True)


print(df)
avgPrice = df.groupby("Rarity").mean()
print(avgPrice)
avgPrice.reset_index( inplace = True)
print(avgPrice)
dataSubset = df[df["Price"] > priceTempest]
print(dataSubset)


data1 = pd.read_html("https://shop.tcgplayer.com/price-guide/pokemon/swsh12-silver-tempest-trainer-gallery")
print(data1)
print(type(data1))
dfTrainer = pd.DataFrame(data1[0])
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.split(")").str[0]
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.split("(").str[0]
print(dfTrainer)
dfTrainer["PRODUCT"] = dfTrainer["PRODUCT"].str.strip()
dfTrainer[[ 'NAME', 'TYPE']] = dfTrainer['PRODUCT'].str.split(' ', n = 1,  expand=True)
dfTrainer["TYPE"] = dfTrainer["TYPE"].str.strip()
dfTrainer = dfTrainer.apply(update_SECRET, axis=1)
print(dfTrainer)
dfTrainer = dfTrainer.apply(update_V2, axis=1)
dfTrainer = dfTrainer.apply(update_VMAX2, axis=1)
dfTrainer = dfTrainer.apply(update_GOLD_AND_BLACK, axis = 1)


print(dfTrainer)
dfTrainer["Number"] = dfTrainer["Number"].str.split("/").str[0]

dfTrainer["Number"] = dfTrainer["Number"].str.replace("TG", "", regex = True)
dfTrainer["Number"] = dfTrainer["Number"].astype(int)
dfTrainer = dfTrainer.apply(update_TRAINERG, axis=1)
dfTrainer.drop(labels = "Number", axis = 1, inplace = True)
dfTrainer.drop(labels = "NAME", axis = 1, inplace = True)
dfTrainer.drop(labels = "TYPE", axis = 1, inplace = True)
dfTrainer.drop(labels = "Unnamed: 5", axis = 1, inplace = True)
dfTrainer.drop(labels = "Listed Median", axis = 1, inplace = True)
dfTrainer.rename(columns = {'Market Price':'Price'}, inplace = True)
dfTrainer["Price"] = dfTrainer["Price"].str.replace("$", "", regex = True)
dfTrainer["Price"] = dfTrainer["Price"].astype(float)
print(dfTrainer)

#Working on thhis dataframe 
dfRate.rename(columns = {'Pull Rate (%)' : 'PullRate'}, inplace = True)
dfRate["PullRate"] = dfRate["PullRate"].str.replace("%", "", regex = True)
dfRate["PullRate"] = dfRate["PullRate"].str.strip()
dfRate.drop(labels = "Pull Rate (Per X Packs)", axis = 1, inplace = True)
dfRate = drop_TRAINERGALLARY(dfRate)
print(dfRate)

dfRate["Card Type"] = dfRate["Card Type"].str.replace("Pokémon ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Pokémon", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.strip()
print(dfRate)
dfRate["PullRate"] = dfRate["PullRate"].astype(float)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Normal ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace(",", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("or ", "", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("V VMAX", "VTrainer VMAXTrainer", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.replace("Gold-and-Black VMAX", "Gold-and-Black", regex = True)
dfRate["Card Type"] = dfRate["Card Type"].str.strip()
dfRate[[ 'NAME', 'TYPE', "OTHER"]] = dfRate['Card Type'].str.split(' ', n = 2,  expand=True)
dfRate = update_TrainerPulls(dfRate)
dfRate.drop(labels = "NAME", axis = 1, inplace = True)
dfRate.drop(labels = "TYPE", axis = 1, inplace = True)
dfRate.drop(labels = "OTHER", axis = 1, inplace = True)
dfRate.drop(1, inplace = True)
dfRate.drop(6, inplace = True)
dfRate.reset_index( inplace = True)
dfRate.drop(labels = "index", axis = 1, inplace = True)
print(dfRate)
dfRate.rename(columns = {'Card Type':'Rarity'}, inplace = True)
dfMerged = dfRate.merge(dfTrainer, on='Rarity')
dfMerged['Weigted'] = dfMerged['PullRate'] * dfMerged['Price']
print(dfMerged)
# dfRate["PullRate"][8] = dfRate["PullRate"][8] / 3


#change the rairtys in trainer set sooo do it ltr


#sort by values to check how Alternative ART looks like which column it falls 
#make a another function that overlaps Full Art with that checks with ART so it doesnt miss any 
#ove Vstar into a different rarity etcc.... 
#When doing the math its 6 commons and 3 uncommons one rare but the last 2 cards rare 

<class 'pandas.core.frame.DataFrame'>
                Card Type Pull Rate (%) Pull Rate (Per X Packs)
0              Ultra Rare        18.98%            1 in 5 Packs
1        Normal Pokémon V        11.55%            1 in 9 Packs
2   Pokémon VMAX or VSTAR         3.72%           1 in 27 Packs
3      Full-Art Pokémon V         2.10%           1 in 48 Packs
4        Full-Art Trainer         1.01%           1 in 99 Packs
5       Alt-Art Pokémon V         0.60%          1 in 168 Packs
6         Trainer Gallery        12.23%            1 in 8 Packs
7           Non-V Pokémon         8.25%           1 in 12 Packs
8     V, VMAX, or Trainer         3.08%           1 in 32 Packs
9     Gold-and-Black VMAX         0.90%          1 in 111 Packs
10           Radiant Rare         4.55%           1 in 22 Packs
11            Secret Rare         2.20%           1 in 45 Packs
12           Rainbow Rare         1.26%           1 in 80 Packs
13              Gold Rare         0.94%          1 in 106 Packs
3.

C:\Users\pcagm\AppData\Local\Temp\ipykernel_6848\2842161006.py:254: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avgPrice = df.groupby("Rarity").mean()


[                    PRODUCT       Rarity     Number Market Price  \
0                   Altaria   Ultra Rare  TG11/TG30        $1.53   
1                Blaziken V   Ultra Rare  TG14/TG30       $13.36   
2             Blaziken VMAX   Ultra Rare  TG15/TG30       $14.15   
3                 Blissey V   Ultra Rare  TG22/TG30       $10.20   
4                   Braixen   Ultra Rare  TG01/TG30        $1.48   
5             Corviknight V   Ultra Rare  TG18/TG30        $7.22   
6          Corviknight VMAX   Ultra Rare  TG19/TG30        $7.25   
7                 Druddigon   Ultra Rare  TG09/TG30        $0.99   
8            Duraludon VMAX   Ultra Rare  TG21/TG30       $16.42   
9   Duraludon VMAX (Secret)  Secret Rare  TG30/TG30        $2.34   
10                  Flaaffy   Ultra Rare  TG03/TG30        $1.89   
11         Friends in Galar   Ultra Rare  TG23/TG30       $17.40   
12                Gardevoir   Ultra Rare  TG05/TG30        $1.22   
13                   Gordie   Ultra Rare  TG24/

C:\Users\pcagm\AppData\Local\Temp\ipykernel_6848\2842161006.py:159: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Card Type": "VTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
C:\Users\pcagm\AppData\Local\Temp\ipykernel_6848\2842161006.py:164: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Card Type": "VMAXTrainer", "PullRate": df["PullRate"][num] / 3 }, ignore_index=True)
C:\Users\pcagm\AppData\Local\Temp\ipykernel_6848\2842161006.py:168: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Card Type": "Trainer", "PullRate": df["PullRate"][num] / 3.0 }, ignore_index=True)
C:\Users\pcagm\AppData\Local\Temp\ipykernel_6848\2842161006.py:171: FutureWarning: The fra